# Extract the HbA1c Values and Diabetes Mentions from MIMIC-III dataset documents

In this notebook I will be developing a python script for extracting the HbA1c values from text documents. The main thing I am looking for is going to be A1c followed by some value.

I will be using PyConText to accomplish this task. I have found when experimenting with PyConText on the MIMIC-III dataset that you sometimes get some very odd things that come back when you use modifiers with numbers, so I am going to use one regular expression to obtain both the mention of HbA1c and the value. I will then remove everything that is not the number to obtain the actual value. 

This notebook is almost identical to the previous notebook, except that in this notebook I am running this on the test dataset. 


First step, import PyConText and define the functions (taken from the PyConText github page and modified with help from Jeff Ferraro) so that I can run the actual text parsing. 

In [16]:
import pyConTextNLP.pyConText as pyConText
# itemData has been rewritten, so that it can take relative local path, where you can redirect it to your customized yml files later
import os
import itemData
import re
import glob
import pandas as pd
from xml.etree import ElementTree
import math


In [2]:
my_targets=itemData.get_items('Yaml_Files/A1c_targets.yml')
my_modifiers=itemData.get_items('Yaml_Files/A1c_modifiers.yml')

The functions *markup_sentence* and *markup_doc* were both ones that we went over in the NLP lab.

In [3]:
## This one is the same, it just doesn't split it into sentences. 
def markup_sentence(s, modifiers, targets, prune_inactive=True):
    """
    """
    markup = pyConText.ConTextMarkup()
    markup.setRawText(s)
    markup.cleanText()
    markup.markItems(my_modifiers, mode="modifier")
    markup.markItems(my_targets, mode="target")
    markup.pruneMarks()
    markup.dropMarks('Exclusion')
    # apply modifiers to any targets within the modifiers scope
    markup.applyModifiers()
    markup.pruneSelfModifyingRelationships()
    if prune_inactive:
        markup.dropInactiveModifiers()
    return markup

def markup_doc(doc_text:str)->pyConText.ConTextDocument:
    rslts=[]
    context = pyConText.ConTextDocument()
    #for s in doc_text.split('.'):
    m = markup_sentence(doc_text, modifiers=my_modifiers, targets=my_targets)
    rslts.append(m)

    for r in rslts:
        context.addMarkup(r)
    return context

def get_output(something):
    context=markup_doc(something)
    output = context.getDocumentGraph()
    return output

Ok, I have figured out how to get the pieces of a node that I can use for every node. I can put these into lists and then add the lists into a dataframe, then transpose the dataframe and I can have something to work with. The next step is going to be reading in the documents and figuring out how to apply 

In [4]:
os.listdir('Text_Files/')

['.DS_Store',
 'Training_Dataset',
 'test_files.txt',
 'Testing_Dataset',
 'list_of_Files.txt']

In [5]:
os.getcwd()

'/Users/david/Documents/David_Sant/Classes/NLP_BMI_6115_Biomedical_Text_Processing/Final_Project/Coding_Portion_NLP_Final_Project'

In [6]:
os.listdir("/Users/david/Documents/David_Sant/MIMIC Database/")

['PROCEDUREEVENTS_MV.csv',
 'CALLOUT.csv',
 'D_CPT.csv',
 'D_ITEMS.csv',
 'CAREGIVERS.csv',
 'MICROBIOLOGYEVENTS.csv',
 'LABEVENTS.csv',
 'INPUTEVENTS_CV.csv',
 'ADMISSIONS.csv',
 'D_LABITEMS.csv',
 'DATETIMEEVENTS.csv',
 'PRESCRIPTIONS.csv',
 'PROCEDURES_ICD.csv',
 'NOTEEVENTS.csv',
 'CHARTEVENTS.csv',
 'TRANSFERS.csv',
 'DIAGNOSES_ICD.csv',
 'SERVICES.csv',
 'DRGCODES.csv',
 'OUTPUTEVENTS.csv',
 'Head.noteevents.csv',
 'PATIENTS.csv',
 'D_ICD_DIAGNOSES.csv',
 'ICUSTAYS.csv',
 'INPUTEVENTS_MV.csv',
 'D_ICD_PROCEDURES.csv',
 'CPTEVENTS.csv']

In [7]:
noteevents = pd.read_csv("/Users/david/Documents/David_Sant/MIMIC Database/NOTEEVENTS.csv")
len(noteevents)

/Users/david/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2083180

In [61]:
len(noteevents)

2083180

In [8]:
noteevents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [9]:
text_df = noteevents[["ROW_ID", "TEXT"]]
text_df.columns = ["Identifier", "Text"]
text_df.head()

,Identifier,Text
0,174,Admission Date: [**2151-7-16**] Dischar...
1,175,Admission Date: [**2118-6-2**] Discharg...
2,176,Admission Date: [**2119-5-4**] D...
3,177,Admission Date: [**2124-7-21**] ...
4,178,Admission Date: [**2162-3-3**] D...


In [18]:
def get_a1c_flag(a):
    try:
        if float(a) < 7.1:
            return "Good"
        elif float(a) >= 7.1 and float(a) < 10.1:
            return "Moderate"
        elif float(a) >= 10.1:
            return "Poor"
        else:
            return "Not Sure"
    except:
        return "Not a value"
output_array = []
count_of_bad_lines = 0
for i in range(len(text_df)):
    raw_text = text_df["Text"][i]
    remove_MIMIC_comments = re.sub(r"\[\*\*.*?\*\*\]", "", raw_text)
    remove_times = re.sub(r"\d{1,2}:\d{2}\s?P?A?\.?M\.?", "", remove_MIMIC_comments)
    cleaned_text = re.sub(r"\s{2,}", r" ", remove_times)
    
    context=markup_doc(cleaned_text)
    
    try:
        root = ElementTree.fromstring(context.getDocumentGraph().getXML())
    except ElementTree.ParseError:
        print("Input Line " + str(i) + " contains a character that is not valid in XML.")
        count_of_bad_lines += 1
        continue
    for node in root.findall('.//node'):
        phrase = node.find('.//phrase').text
        tmp1 =  re.sub(r"[A|a]1[C|c]", "", phrase)
        A1c_Value = re.sub(r"[^\d{1,2}\.?\d{0,1}]", "", tmp1)
        A1c_Flag = get_a1c_flag(A1c_Value)
        literal = node.find('.//literal').text
        Start = node.find('.//spanStart').text
        Stop = node.find('.//spanStop').text
        Node_ID = node.find('.//id').text
        category = node.find('.//category').text
        try:
            modified_by = node.find('.//modifyingNode').text
        except:
            modified_by = "None"
        try:
            modifying_category = node.find('.//modifyingCategory').text
        except:
            modifying_category = "None"
        try:
            node_modified = node.find('.//modifiedNode').text
        except:
            node_modified = "None"
        output_array.append([text_df["Identifier"][i], Start, Stop, phrase, literal, A1c_Value, A1c_Flag, Node_ID,
                             modifying_category, modified_by, node_modified])
print("Number of documents containing invalid text = ", count_of_bad_lines)
            
#output_array

Input Line 1241030 contains a character that is not valid in XML.
Input Line 1241403 contains a character that is not valid in XML.
Input Line 1242699 contains a character that is not valid in XML.
Input Line 1243701 contains a character that is not valid in XML.
Input Line 1243922 contains a character that is not valid in XML.
Input Line 1245238 contains a character that is not valid in XML.
Input Line 1246795 contains a character that is not valid in XML.
Input Line 1246890 contains a character that is not valid in XML.
Input Line 1247559 contains a character that is not valid in XML.
Input Line 1258525 contains a character that is not valid in XML.
Input Line 1261567 contains a character that is not valid in XML.
Input Line 1263152 contains a character that is not valid in XML.
Input Line 1263893 contains a character that is not valid in XML.
Input Line 1263895 contains a character that is not valid in XML.
Input Line 1263927 contains a character that is not valid in XML.
Input Line

Input Line 1396786 contains a character that is not valid in XML.
Input Line 1398240 contains a character that is not valid in XML.
Input Line 1400867 contains a character that is not valid in XML.
Input Line 1402199 contains a character that is not valid in XML.
Input Line 1402713 contains a character that is not valid in XML.
Input Line 1404089 contains a character that is not valid in XML.
Input Line 1405116 contains a character that is not valid in XML.
Input Line 1405755 contains a character that is not valid in XML.
Input Line 1406276 contains a character that is not valid in XML.
Input Line 1406307 contains a character that is not valid in XML.
Input Line 1407131 contains a character that is not valid in XML.
Input Line 1407332 contains a character that is not valid in XML.
Input Line 1408142 contains a character that is not valid in XML.
Input Line 1409235 contains a character that is not valid in XML.
Input Line 1411997 contains a character that is not valid in XML.
Input Line

Input Line 1547073 contains a character that is not valid in XML.
Input Line 1549290 contains a character that is not valid in XML.
Input Line 1550760 contains a character that is not valid in XML.
Input Line 1550847 contains a character that is not valid in XML.
Input Line 1554919 contains a character that is not valid in XML.
Input Line 1555752 contains a character that is not valid in XML.
Input Line 1556538 contains a character that is not valid in XML.
Input Line 1561347 contains a character that is not valid in XML.
Input Line 1561439 contains a character that is not valid in XML.
Input Line 1564107 contains a character that is not valid in XML.
Input Line 1564117 contains a character that is not valid in XML.
Input Line 1565215 contains a character that is not valid in XML.
Input Line 1566620 contains a character that is not valid in XML.
Input Line 1567106 contains a character that is not valid in XML.
Input Line 1571183 contains a character that is not valid in XML.
Input Line

Input Line 1638698 contains a character that is not valid in XML.
Input Line 1638713 contains a character that is not valid in XML.
Input Line 1638860 contains a character that is not valid in XML.
Input Line 1638976 contains a character that is not valid in XML.
Input Line 1639158 contains a character that is not valid in XML.
Input Line 1639160 contains a character that is not valid in XML.
Input Line 1639807 contains a character that is not valid in XML.
Input Line 1640110 contains a character that is not valid in XML.
Input Line 1640125 contains a character that is not valid in XML.
Input Line 1641039 contains a character that is not valid in XML.
Input Line 1641471 contains a character that is not valid in XML.
Input Line 1643271 contains a character that is not valid in XML.
Input Line 1643723 contains a character that is not valid in XML.
Input Line 1644093 contains a character that is not valid in XML.
Input Line 1644106 contains a character that is not valid in XML.
Input Line

Input Line 1968593 contains a character that is not valid in XML.
Input Line 1971236 contains a character that is not valid in XML.
Input Line 1976176 contains a character that is not valid in XML.
Input Line 1979230 contains a character that is not valid in XML.
Input Line 1984383 contains a character that is not valid in XML.
Input Line 1988990 contains a character that is not valid in XML.
Input Line 1991117 contains a character that is not valid in XML.
Input Line 1993933 contains a character that is not valid in XML.
Input Line 2000880 contains a character that is not valid in XML.
Input Line 2002131 contains a character that is not valid in XML.
Input Line 2007838 contains a character that is not valid in XML.
Input Line 2014665 contains a character that is not valid in XML.
Input Line 2019691 contains a character that is not valid in XML.
Input Line 2019879 contains a character that is not valid in XML.
Input Line 2020174 contains a character that is not valid in XML.
Input Line

In [19]:
len(output_array)

11489

In [17]:
type(output_array)

list

In [19]:
test_df = pd.DataFrame(output_array, columns=("Identifier", "Start", "Stop", "Phrase", "Annotation_Type", "A1c_Value", "A1c_Flag", "Node_ID", "Modifying_Category", "Modified_By", "Node_Modified"))
test_df.tail() # New dataframe name

,Identifier,Start,Stop,Phrase,Annotation_Type,A1c_Value,A1c_Flag,Node_ID,Modifying_Category,Modified_By,Node_Modified
11358,871855,379,387,A1C 16.5,A1C_COLON_OR_SPACE,16.5,Poor,104369333023302219760999297321664009023,None,None,None
11359,961497,61,67,A1C 13,A1C_COLON_OR_SPACE,13,Poor,139753657991440482938734613806757466943,None,None,None
11360,1212092,311,318,A1C 7.2,A1C_COLON_OR_SPACE,7.2,Moderate,234934268417259419230930970163800462143,None,None,None
11361,1212344,279,286,A1C 7.2,A1C_COLON_OR_SPACE,7.2,Moderate,234997228661164629672086431734813971263,None,None,None
11362,1211386,330,338,A1c 12.4,A1C_COLON_OR_SPACE,12.4,Poor,235891431771410498119832333982204711743,None,None,None


I had put in f/u as a modifier so that it would have a modifier file, but all of my targets require a value. In other words, if they put f/u and a value it is either a mistake typing or it refers to something else. As such, I can get rid of all of the modifier columns and the rows about node ID, modifying node, modified by, and node modified. 

In [20]:
modifier_columns = test_df[test_df["Node_Modified"]!="None"]
len(modifier_columns)

848

In [21]:
modifier_columns = test_df[test_df["Node_Modified"]!="None"]
A1c_Value_Results = test_df[["Identifier", "Start", "Stop", "Phrase", "Annotation_Type", "A1c_Value", "A1c_Flag"]].drop(modifier_columns.index, axis = 0)
len(A1c_Value_Results)

10515

In [22]:
A1c_Value_Results.head()

,Identifier,Start,Stop,Phrase,Annotation_Type,A1c_Value,A1c_Flag
0,242,3020,3027,A1c-5.4,A1C_DASH,5.4,Good
1,193,3764,3771,A1c-9.4,A1C_DASH,9.4,Moderate
2,193,6900,6907,A1c 9.4,A1C_COLON_OR_SPACE,9.4,Moderate
3,196,2158,2169,A1C was 5.9,A1C_IS_OR_WAS,5.9,Good
4,198,2991,2998,A1c 6.3,A1C_COLON_OR_SPACE,6.3,Good


In [17]:
A1c_Value_Results["A1c_Value"] = A1c_Value_Results["A1c_Value"].apply(pd.to_numeric)
A1c_Value_Results = A1c_Value_Results.groupby("Identifier").apply(lambda x: x.loc[x.A1c_Value.idxmax()])
A1c_Value_Results.to_csv("Output_Files/A1c_Results_Test_Dataset.csv") #New output file names
modifier_columns.to_csv("Output_Files/Modifier_Columns_to_A1c_Test_Dataset.csv") # I know this isn't needed, but I am going to save it for just in case

In [18]:
A1c_Value_Results.head()

,Identifier,Start,Stop,Phrase,Annotation_Type,A1c_Value,A1c_Flag
Identifier,,,,,,,
26293,26293,487,498,A1C was 5.7,A1C_IS_OR_WAS,5.7,Good
3010,3010,1143,1153,A1C of 6.7,A1C_IN_OF,6.7,Good
405628,405628,173,180,A1C:6.4,A1C_COLON_OR_SPACE,6.4,Good
410226,410226,560,567,A1c 7.0,A1C_COLON_OR_SPACE,7.0,Good
420089,420089,129,137,A1c: 6.2,A1C_COLON_OR_SPACE,6.2,Good


In [23]:
A1c_Value_Results.to_csv("Output_Files/MIMIC_Results_A1c.csv")

In [25]:
A1c_Value_Results.groupby("A1c_Flag").count()

,Identifier,Start,Stop,Phrase,Annotation_Type,A1c_Value
A1c_Flag,,,,,,
Good,7160,7160,7160,7160,7160,7160
Moderate,2428,2428,2428,2428,2428,2428
Poor,927,927,927,927,927,927


It looks like it read all the way through the entire dataset. I have A1c values from 10,515 documents.
7,160 of them have a good A1c (below 7.0%)
2,428 of them have moderate A1c value (between 7 and 10%)
927 of them have high A1c values above 10%. 

In [26]:
my_targets=itemData.get_items('Yaml_Files/Diabetes_targets.yml')
my_modifiers=itemData.get_items('Yaml_Files/Diabetes_modifiers.yml')

In [14]:

output_array = []
count_of_bad_lines = 0
for i in range(len(text_df)):
    raw_text = text_df["Text"][i]
    remove_MIMIC_comments = re.sub(r"\[\*\*.*?\*\*\]", "", raw_text)
    remove_times = re.sub(r"\d{1,2}:\d{2}\s?P?A?\.?M\.?", "", remove_MIMIC_comments)
    cleaned_text = re.sub(r"\s{2,}", r" ", remove_times)
    
    context=markup_doc(cleaned_text)
    try:
        root = ElementTree.fromstring(context.getDocumentGraph().getXML())
    except ElementTree.ParseError:
        print("Line Number "+str(i)+ " contains a character that is not valid in XML.")
        count_of_bad_lines += 1
        continue
    for node in root.findall('.//node'):
        phrase = node.find('.//phrase').text
        #tmp1 =  re.sub(r"[A|a]1[C|c]", "", phrase)
        #A1c_Value = re.sub(r"[^\d{1,2}\.?\d{0,1}]", "", tmp1)
        #A1c_Flag = get_a1c_flag(A1c_Value)
        literal = node.find('.//literal').text
        Start = node.find('.//spanStart').text
        Stop = node.find('.//spanStop').text
        Node_ID = node.find('.//id').text
        category = node.find('./category').text #This picks up target or modifier, not useful
        try:
            modified_by = node.find('.//modifyingNode').text
        except:
            modified_by = "None"
        try:
            modifying_category = node.find('.//modifyingCategory').text
        except:
            modifying_category = "None"
        try:
            node_modified = node.find('.//modifiedNode').text
        except:
            node_modified = "None"
        output_array.append([text_df["Identifier"][i], Start, Stop, phrase, literal, Node_ID,
                             modifying_category, modified_by, node_modified])
print("Number of documents containing invalid text = ", count_of_bad_lines)

KeyboardInterrupt: 

This gives a parse error for both of these, but the number of documents containing mentions actually matches what I got using SQL, so I believe both systems have finished correctly. 

In [28]:
output_array
train_df = pd.DataFrame(output_array, columns=("Identifier", "Start", "Stop", "Phrase", "Annotation_Type", "Node_ID", "Modifying_Category", "Modified_By", "Node_Modified"))
train_df


,Identifier,Start,Stop,Phrase,Annotation_Type,Node_ID,Modifying_Category,Modified_By,Node_Modified
0,179,2417,2420,not,NOT,333229351697215573041743895114373681983,None,None,333229664648457504385877389612977509183
1,179,2560,2568,diabetes,DIABETES_OR_DIABETIC,333229664648457504385877389612977509183,['hypothetical_diabetes'],333228793138669847478163860629523813183,None
2,179,2638,2640,no,NOT,333229363581439950181394534145966232383,None,None,333229664648457504385877389612977509183
3,179,2839,2845,likely,HYPOTHETICAL_DIABETES,333228793138669847478163860629523813183,None,None,333229664648457504385877389612977509183
4,179,3305,3307,no,NOT,333229388142170329603339188144590836543,None,None,333229664648457504385877389612977509183
5,179,3766,3773,with no,NOT,333229407949210958169423586530578420543,None,None,333229664648457504385877389612977509183
6,179,5057,5059,no,NOT,333229451524700341014809262979751105343,None,None,333229664648457504385877389612977509183
7,179,5530,5533,not,NOT,333229470539459344438250285430299185983,None,None,333229664648457504385877389612977509183
8,179,5932,5935,not,NOT,333229487969655097576404556009968259903,None,None,333229664648457504385877389612977509183
9,179,6700,6703,not,NOT,333229514907230352426279337814911374143,None,None,333229664648457504385877389612977509183


In [29]:
len(train_df)


1479273

In [30]:
modifier_columns = train_df[train_df["Node_Modified"]!="None"]
Diabetes_Results = train_df.drop(modifier_columns.index, axis = 0)
print(len(Diabetes_Results))
print(len(modifier_columns))
modifier_columns.head()

211326
1267947


,Identifier,Start,Stop,Phrase,Annotation_Type,Node_ID,Modifying_Category,Modified_By,Node_Modified
0,179,2417,2420,not,NOT,333229351697215573041743895114373681983,None,None,333229664648457504385877389612977509183
2,179,2638,2640,no,NOT,333229363581439950181394534145966232383,None,None,333229664648457504385877389612977509183
3,179,2839,2845,likely,HYPOTHETICAL_DIABETES,333228793138669847478163860629523813183,None,None,333229664648457504385877389612977509183
4,179,3305,3307,no,NOT,333229388142170329603339188144590836543,None,None,333229664648457504385877389612977509183
5,179,3766,3773,with no,NOT,333229407949210958169423586530578420543,None,None,333229664648457504385877389612977509183


In [31]:
node_locations = Diabetes_Results[["Start", "Stop", "Node_ID"]]
node_locations.head()

,Start,Stop,Node_ID
1,2560,2568,333229664648457504385877389612977509183
10,500,517,333257280416783476363388995292306625343
15,2573,2590,333257304977513855785333649290931229503
16,2936,2944,333257312900330107211767408645326263103
46,11800,11808,333257380244268244336454363157684048703


In [32]:
node_locations.rename(columns={"Start":"Node_Start", "Stop":"Node_Stop", "Node_ID":"Node_Modified"}, inplace = True)
node_locations.head()

/Users/david/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Node_Start,Node_Stop,Node_Modified
1,2560,2568,333229664648457504385877389612977509183
10,500,517,333257280416783476363388995292306625343
15,2573,2590,333257304977513855785333649290931229503
16,2936,2944,333257312900330107211767408645326263103
46,11800,11808,333257380244268244336454363157684048703


In [33]:
modifier_columns = pd.merge(modifier_columns, node_locations, on='Node_Modified') #, how='right'
modifier_columns = modifier_columns[pd.notnull(modifier_columns['Identifier'])] # Drop the ones that weren't modifier nodes
len(modifier_columns)




1267947

In [34]:
distance_negation = 40
distance_other = 60
distance_hypothetical = 100
distance_type = 100

def out_of_range(anno_type, distance):
    if anno_type == " NOT " or anno_type == " DENIES ":
        if distance <= distance_negation:
            return "Keep"
        else:
            return "Discard"
    elif anno_type == " DIABETES_IN_OTHER ":
        if distance <= distance_other:
            return "Keep"
        else:
            return "Discard"
    elif anno_type == " HYPOTHETICAL_DIABETES ":
        if distance <= distance_hypothetical:
            return "Keep"
        else: 
            return "Discard"
    elif anno_type == " DIABETES_TYPE_1 " or anno_type == " DIABETES_TYPE_2 " or anno_type == " DIABETES_GESTATIONAL " or anno_type == " DIABETES_INSIPIDUS ":
        if distance <= distance_type:
            return "Keep"
        else:
            return "Discard"
    else:
        return "Didn't Work"
        

In [37]:
modifier_columns["Distance"] = modifier_columns.apply(lambda x: max((int(x["Start"]) - int(x["Node_Stop"])), (int(x["Node_Start"])-int(x["Stop"]))), axis = 1)
modifier_columns
modifier_columns.head()

,Identifier,Start,Stop,Phrase,Annotation_Type,Node_ID,Modifying_Category,Modified_By,Node_Modified,Node_Start,Node_Stop,Distance
0,179,2417,2420,not,NOT,333229351697215573041743895114373681983,None,None,333229664648457504385877389612977509183,2560,2568,140
1,179,2638,2640,no,NOT,333229363581439950181394534145966232383,None,None,333229664648457504385877389612977509183,2560,2568,70
2,179,2839,2845,likely,HYPOTHETICAL_DIABETES,333228793138669847478163860629523813183,None,None,333229664648457504385877389612977509183,2560,2568,271
3,179,3305,3307,no,NOT,333229388142170329603339188144590836543,None,None,333229664648457504385877389612977509183,2560,2568,737
4,179,3766,3773,with no,NOT,333229407949210958169423586530578420543,None,None,333229664648457504385877389612977509183,2560,2568,1198


In [38]:
modifier_columns["Keep"] = modifier_columns.apply(lambda x: out_of_range(x["Annotation_Type"], int(x["Distance"])), axis = 1)
modifier_columns.head()



,Identifier,Start,Stop,Phrase,Annotation_Type,Node_ID,Modifying_Category,Modified_By,Node_Modified,Node_Start,Node_Stop,Distance,Keep
0,179,2417,2420,not,NOT,333229351697215573041743895114373681983,None,None,333229664648457504385877389612977509183,2560,2568,140,Discard
1,179,2638,2640,no,NOT,333229363581439950181394534145966232383,None,None,333229664648457504385877389612977509183,2560,2568,70,Discard
2,179,2839,2845,likely,HYPOTHETICAL_DIABETES,333228793138669847478163860629523813183,None,None,333229664648457504385877389612977509183,2560,2568,271,Discard
3,179,3305,3307,no,NOT,333229388142170329603339188144590836543,None,None,333229664648457504385877389612977509183,2560,2568,737,Discard
4,179,3766,3773,with no,NOT,333229407949210958169423586530578420543,None,None,333229664648457504385877389612977509183,2560,2568,1198,Discard


In [39]:
modifier_columns = modifier_columns[modifier_columns["Keep"] == "Keep"]
len(modifier_columns)

53520

In [40]:
def get_negated(value):
    if value == " DENIES ":
        return "Negated_Diabetes"
    elif value == " NOT ":
        return "Negated_Diabetes"
    else:
        return ""
    
def get_other(value):
    if value == " DIABETES_IN_OTHER ":
        return "Diabetes_in_other"
    else:
        return ""
    
def get_type(value):
    if value == " DIABETES_TYPE_1 ":
        return "Diabetes_Type_1"
    elif value == " DIABETES_TYPE_2 ":
        return "Diabetes_Type_2"
    elif value == " DIABETES_GESTATIONAL ":
        return "Diabetes_Gestational"
    elif value == " INSIPIDUS ":
        return "Diabetes_Insipidus"
    else:
        return "No_Type"
    
    
    

def get_hypothetical(value):
    if value == " HYPOTHETICAL_DIABETES ":
        return "Diabetes_Hypothetical"
    else:
        return ""

modifier_columns["Negated"] = modifier_columns["Annotation_Type"].apply(get_negated)
modifier_columns["Diabetes_in_other"] = modifier_columns["Annotation_Type"].apply(get_other)
modifier_columns["Type"] = modifier_columns["Annotation_Type"].apply(get_type)
modifier_columns["Hypothetical"] = modifier_columns["Annotation_Type"].apply(get_hypothetical)

/Users/david/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/david/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/david/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [41]:
def max_len(s):
    return max(s, key=len)
def max_val(s):
    return max(s, key=int)
subset = modifier_columns.groupby("Node_Modified").agg({'Diabetes_in_other': max_len, "Hypothetical": max_len, "Negated": max_len, "Type": max_len, "Distance": max_val})


In [42]:
subset = subset.reset_index()
subset.rename(columns={"Node_Modified":"Node_ID"}, inplace = True)
subset

,Node_ID,Diabetes_in_other,Hypothetical,Negated,Type,Distance
0,100002643531914455145824286033507275583,,,,Diabetes_Type_1,1
1,100012921326069664498573885558095369023,,,,Diabetes_Type_2,7
2,100014181053853641301541622906905711423,,,,Diabetes_Type_2,7
3,100019487756178846726873638480699216703,,,,Diabetes_Type_2,92
4,100048306049551833431042395082582414143,,,,Diabetes_Type_1,3
5,10006215416225635528702095234290963263,,,,Diabetes_Type_1,1
6,100065198919909217696214644594934567743,,,,Diabetes_Type_1,7
7,100080925551708347361239443994464244543,,Diabetes_Hypothetical,Negated_Diabetes,No_Type,79
8,100112769410159891272669525256945816383,,,,Diabetes_Type_1,83
9,100123605921314733365888605195122266943,,,,Diabetes_Type_2,1


In [43]:
Final_table = pd.merge(Diabetes_Results, subset, on='Node_ID', how = "left") # how='right'
Final_table

,Identifier,Start,Stop,Phrase,Annotation_Type,Node_ID,Modifying_Category,Modified_By,Node_Modified,Diabetes_in_other,Hypothetical,Negated,Type,Distance
0,179,2560,2568,diabetes,DIABETES_OR_DIABETIC,333229664648457504385877389612977509183,['hypothetical_diabetes'],333228793138669847478163860629523813183,None,NaN,NaN,NaN,NaN,NaN
1,184,500,517,diabetes mellitus,DIABETES_OR_DIABETIC,333257280416783476363388995292306625343,['diabetes_in_other'],333253075778198844354992905914862293823,None,NaN,NaN,NaN,NaN,NaN
2,184,2573,2590,Diabetes mellitus,DIABETES_OR_DIABETIC,333257304977513855785333649290931229503,['diabetes_in_other'],333253075778198844354992905914862293823,None,NaN,NaN,NaN,NaN,NaN
3,184,2936,2944,Diabetes,DIABETES_OR_DIABETIC,333257312900330107211767408645326263103,['diabetes_in_other'],333253075778198844354992905914862293823,None,NaN,NaN,NaN,NaN,NaN
4,184,11800,11808,Diabetes,DIABETES_OR_DIABETIC,333257380244268244336454363157684048703,['diabetes_in_other'],333253101131210848919580935848926401343,None,NaN,NaN,NaN,NaN,NaN
5,184,12803,12820,Diabetes mellitus,DIABETES_OR_DIABETIC,333257392920774246618748378124716102463,['diabetes_in_other'],333253101131210848919580935848926401343,None,NaN,NaN,NaN,NaN,NaN
6,227,2322,2330,diabetes,DIABETES_OR_DIABETIC,333306600740230231056184352336830288703,['diabetes_in_other'],333303887967945742645265149391970784063,None,Diabetes_in_other,,,No_Type,28.0
7,230,1327,1335,diabetes,DIABETES_OR_DIABETIC,333338368064271950484985859719157011263,['diabetes_in_other'],333337021977790833133890145407440802623,None,Diabetes_in_other,,Negated_Diabetes,No_Type,38.0
8,231,1747,1755,diabetes,DIABETES_OR_DIABETIC,333342651138737471615076166705112175423,['diabetes_in_other'],333340888312121529233564710352217199423,None,Diabetes_in_other,,Negated_Diabetes,No_Type,38.0
9,232,1684,1692,diabetes,DIABETES_OR_DIABETIC,333349627970728477732644654185378763583,['diabetes_in_other'],333346203729544611227973861215845241663,None,Diabetes_in_other,,Negated_Diabetes,No_Type,38.0


In [44]:
def get_new_type(anno_type, modify_type):
    if anno_type == " DMII " or anno_type == " DM2 " or anno_type == " T2DM " or anno_type == " NIDDM ":
        return "Diabetes_Type_2"
    elif anno_type == " DMI " or anno_type == " DM1 " or anno_type == " T1DM " or anno_type == " IDDM ":
        return "Diabetes_Type_1"
    elif anno_type == " GDM ":
        return "Diabetes_Gestational"
    else:
        if modify_type == "Diabetes_Type_1":
            return "Diabetes_Type_1"
        elif modify_type == "Diabetes_Type_2":
            return "Diabetes_Type_2"
        elif modify_type == "Diabetes_Gestational":
            return "Diabetes_Gestational"
        elif modify_type == "Diabetes_Insipidus":
            return "Diabetes_Insipidus"
        else:
            return "Diabetes_Type_Not_Specified"
        
        

Final_table["Diabetes_Type"] = Final_table.apply(lambda x: get_new_type(x["Annotation_Type"], x["Type"]), axis = 1)


In [45]:
Final_table["Diabetes_Negated"] = Final_table.apply(lambda x: "Negated_Diabetes" if x["Negated"] == "Negated_Diabetes" else None, axis = 1)
Final_table["Diabetes_Hypothetical"] = Final_table.apply(lambda x: "Diabetes_Hypothetical" if x["Hypothetical"] == "Diabetes_Hypothetical" else None, axis = 1)
Final_table["Diabetes_In_Other_Person"] = Final_table.apply(lambda x: "Diabetes_in_other" if x["Diabetes_in_other"] == "Diabetes_in_other" else None, axis = 1)


Final_table.head()

,Identifier,Start,Stop,Phrase,Annotation_Type,Node_ID,Modifying_Category,Modified_By,Node_Modified,Diabetes_in_other,Hypothetical,Negated,Type,Distance,Diabetes_Type,Diabetes_Negated,Diabetes_Hypothetical,Diabetes_In_Other_Person
0,179,2560,2568,diabetes,DIABETES_OR_DIABETIC,333229664648457504385877389612977509183,['hypothetical_diabetes'],333228793138669847478163860629523813183,None,NaN,NaN,NaN,NaN,NaN,Diabetes_Type_Not_Specified,None,None,None
1,184,500,517,diabetes mellitus,DIABETES_OR_DIABETIC,333257280416783476363388995292306625343,['diabetes_in_other'],333253075778198844354992905914862293823,None,NaN,NaN,NaN,NaN,NaN,Diabetes_Type_Not_Specified,None,None,None
2,184,2573,2590,Diabetes mellitus,DIABETES_OR_DIABETIC,333257304977513855785333649290931229503,['diabetes_in_other'],333253075778198844354992905914862293823,None,NaN,NaN,NaN,NaN,NaN,Diabetes_Type_Not_Specified,None,None,None
3,184,2936,2944,Diabetes,DIABETES_OR_DIABETIC,333257312900330107211767408645326263103,['diabetes_in_other'],333253075778198844354992905914862293823,None,NaN,NaN,NaN,NaN,NaN,Diabetes_Type_Not_Specified,None,None,None
4,184,11800,11808,Diabetes,DIABETES_OR_DIABETIC,333257380244268244336454363157684048703,['diabetes_in_other'],333253101131210848919580935848926401343,None,NaN,NaN,NaN,NaN,NaN,Diabetes_Type_Not_Specified,None,None,None


In [53]:
Relevant_Columns = Final_table[["Identifier", "Start", "Stop", "Phrase", "Annotation_Type", "Distance",
                             "Diabetes_Type", "Diabetes_Negated", "Diabetes_Hypothetical", "Diabetes_In_Other_Person"]]





In [54]:
Relevant_Columns.to_csv("Output_Files/MIMIC_Diabetes_mention.csv")

In [55]:
Relevant_Columns.groupby("Diabetes_Type").count()

,Identifier,Start,Stop,Phrase,Annotation_Type,Distance,Diabetes_Negated,Diabetes_Hypothetical,Diabetes_In_Other_Person
Diabetes_Type,,,,,,,,,
Diabetes_Gestational,565,565,565,565,565,550,30,20,217
Diabetes_Type_1,21531,21531,21531,21531,21531,12064,1447,788,182
Diabetes_Type_2,41139,41139,41139,41139,41139,17334,1786,1458,361
Diabetes_Type_Not_Specified,148091,148091,148091,148091,148091,10458,5179,3874,2708


In [56]:
Relevant_Columns.groupby("Diabetes_Negated").count()

,Identifier,Start,Stop,Phrase,Annotation_Type,Distance,Diabetes_Type,Diabetes_Hypothetical,Diabetes_In_Other_Person
Diabetes_Negated,,,,,,,,,
Negated_Diabetes,8442,8442,8442,8442,8442,8442,8442,876,842


In [57]:
Relevant_Columns.groupby("Diabetes_Hypothetical").count()

,Identifier,Start,Stop,Phrase,Annotation_Type,Distance,Diabetes_Type,Diabetes_Negated,Diabetes_In_Other_Person
Diabetes_Hypothetical,,,,,,,,,
Diabetes_Hypothetical,6140,6140,6140,6140,6140,6140,6140,876,44


In [58]:

Relevant_Columns.groupby("Diabetes_In_Other_Person").count()


,Identifier,Start,Stop,Phrase,Annotation_Type,Distance,Diabetes_Type,Diabetes_Negated,Diabetes_Hypothetical
Diabetes_In_Other_Person,,,,,,,,,
Diabetes_in_other,3468,3468,3468,3468,3468,3468,3468,842,44


In [59]:
len(Relevant_Columns)

211326

In [60]:
len(Relevant_Columns.groupby("Identifier"))

102169

OK, it finally ran!

Out of the 2,083,180 documents, 102,169 of them have a mention of diabetes. There are a total of 211,326 mentions of diabetes (just over 2 per document that mentions diabetes on average). Of these 211,326 mentions, 565 of them are gestational diabetes, 21,531 are Type 1 diabetes and 41,139 are Type 2 diabetes. I don't trust those numbers because in my test dataset it guessed less than 2/3 of the types correctly. Additionally, 8,842 of those mentions are negated, 6,140 are hypothetical and 3,468 are in a person other than the patient. Again, I think these numbers are underestimates, just like the types. 

As for the A1c, there are 10,515 documents that give an HbA1c value. 7,160 (68.1%) of them have a good A1c (7.0% or below). This is a good sign. Another 2,428 (23.1%) are fair management of diabetes and only 927 (8.8%) are high (above 10%, poor management of diabetes). This isn't bad at all. 